In [ ]:
# numerical tools
import numpy as np
import scipy
# plotting
import matplotlib.pyplot as plt



# Background: What is a computer?  
Modern hardware is optimized for very particular numerical operations, in general computers operate as *Turing Machines* which means they have a state (which is associated with the computer's memory), and actions which modify the state (these are called instructions).  An operating system is characterized by the set of instructions, and the default memory layout e.g. for an *x86-64* system the *instruction set* is x86, and memory addresses are most-naturally 64 bits (8 bytes).  As an aside, nowdays smaller or reduced instruction sets have become somewhat popular, for example new macbook computers and many smartphone use a type of reduced instruction set built by a company ARM (Advanced (Reduced instruction set computer) Machines) .  Programming languages allow people to write code which is converted to machine instructions by either a *compiler* (e.g. C, C++, Java(usually)) or an *interpreter* (Python(usually)).  Usually, but not always a program with fewer instructions will run faster than a program with many instructions.  


## Why do we care?
Machines can only perform certain operations efficiently, because they have a limited number of operations which can be performed in a small number of instructions.  

In [ ]:
# Example Addition can be performed in a single instruction on modern machines
# We can time how long it takes to perform the operation with the ipython magic command %timeit
%timeit 1032421.1 + 123012.4

In [ ]:
# Trig functions cannot be performed in a single instruction usually
# They usually take much much longer than other operations
%timeit np.sin(1.3)

## Vectorization
Most modern machines have efficient strategies for evaluating multiple floating point operations at once (called vectorization).  Generally it is faster to vectorize your code if you can.  In python, almost every `numpy` operation is vectorized, therefore you should always use numpy arrays, `np.array([...])` (and not python lists `[...]`!) when writing numeric code.  

In [ ]:
# For example
x = np.arange(10000)
%timeit y = x ** 2

In [ ]:
# Compared to
x = np.arange(10000)
%timeit y = [x[i]**2 for i in range(len(x))]

In [ ]:
# The second approach is almost 1000x slower!
# Some operations may be surprisingly slow: for example compare the following to the cell above
x = np.arange(10000)
%timeit y = x ** 2.1

By just changing the exponent to 2.1 from 2 we have made the code run almost 100x slower.  This is because computers can do $x^2$ efficiently by rewriting it as the instruction $x * x$, but in order to evaluate $x^{2.1}$ it has to run a relativily sophisticated algorithm that rewrites the expression as $e^{2.1 \ln(x)}$ and evaluate both a log and and an exponential function.

In [ ]:
# Be careful what ways you call functions! In many cases there is a special version of a function which is faster than a more general version
%timeit np.sqrt(x)

In [ ]:
%timeit x**(.5)

Writing efficient code is good for you and also for people who use your code (who spend less time waiting for code to run), but also is good for the planet.  A CPU takes somewhere around 10 watts to run.  A single CPU running all day will require the emission of about 100 g of $\rm{CO}_2$; many high performance codes require 10's of thousands of CPU hours to run.  Well written, high-performance code can easily prevent a ton of $\rm{CO}_2$ from entering the atmosphere over the lifetime of the code!

# Solving Linear Equations
The simplest equation we can solve on a computer is something of the form `b*x + c = y`

In [ ]:
# We solve equations by finding a value for the unknowns that make them true
# Set up a specific problem
b = 2.0
c = 3.0
y = 5.0
# Solving step
x = (y - c) / b

In [ ]:
# In this case x is
x

More generally, if $y=0$ then 
$$x=-\frac{c}{b}$$

In [ ]:
# It turns out this is basically the only problem that computers can solve (with a few caveats)
# But what about quadratic equations! Surely computers can solve them (we'll just ask ChatGPT to write us a code that solves them)
import numpy as np

def solve_quadratic(a, b, c):
    # Calculate the discriminant
    discriminant = b**2 - 4*a*c
    
    # Check if the discriminant is positive, negative, or zero
    if discriminant > 0:
        # Two real and different roots
        root1 = (-b + np.sqrt(discriminant)) / (2*a)
        root2 = (-b - np.sqrt(discriminant)) / (2*a)
        return root1, root2
    elif discriminant == 0:
        # One real root
        root = -b / (2*a)
        return root
    else:
        # Two complex roots
        real_part = -b / (2*a)
        imaginary_part = np.sqrt(-discriminant) / (2*a)
        root1 = complex(real_part, imaginary_part)
        root2 = complex(real_part, -imaginary_part)
        return root1, root2

# Example usage:
a = 1
b = -3
c = 2

roots = solve_quadratic(a, b, c)
print("The roots of the quadratic equation are:", roots)

The Quadratic formula can actually be seen to reduce to the expression for a linear function above when a
(the coefficient on the quadratic term) is small compared to b^2/c.  Concretely
$$x = \frac{-b \pm \sqrt{b^2 - 4 a c}}{2a}$$
Taylor exapandng assuming $4ac \ll b^2$

$$\sqrt{b^2 - 4ac} =  b\sqrt{1-\frac{4ac}{b^2}} \approx b(1 - \frac{2ac}{b^2})$$
So the entire expression becomes 
$$x = \frac{b \pm (b -  \frac{2ac}{b})}{2a}$$
The "+" root will go to infinity as $a$ gets small, so we want the "-" root, which will become the solution of the equation in the limit.  
This becomes 
$$x = \frac{2ac}{2ab} = -\frac{c}{b}$$
Exactly what we expected! Let's use the code ChatGPT wrote to show how one of the roots converges to this value as a get's small!

In [ ]:
# We can now use Chat GPT's code to see this limiting happen in practice!
# -b/c = -2.0, we just have to make a really small!
b = 2.0
c = 1.0
a = 1e-1
roots = solve_quadratic(a, b, c)
print("The roots of the quadratic equation are:", roots)

In [ ]:
# Uh oh

## Solving Linear systems
One of the most important operations modern computers perform today is solving systems of the form 
$$Ax=b$$ with $A$ a matrix and $x$ and $b$ vectors

In [ ]:
# For something like this, it's always best to find a library to solve the problem
# We'll use the numpy linalg library

# make a matrix out of random gaussian draws
A = np.random.randn(5,5)
b = np.array([1,0,0,0,0])

In [ ]:
x = np.linalg.solve(A, b)
print("x = ", x)

What about the matrix inverse?  
many will have learned to solve the above system as 
$$x = A^{-1}b$$
Generally, though, this is slower than using a function like `np.linalg.solve`

In [ ]:
# Let's demonstrate on a bigger problem
A = np.random.randn(2000,2000)
b = np.zeros(2000)
b[0] = 1.0


In [ ]:
%timeit x = np.linalg.solve(A, b)

In [ ]:
# Hm why does this seem so fast!!?
%timeit x = A**-1 * b 

In [ ]:
# This is a trap! The above computation is not computing the right thing (it's doing everything pointwise)! 
# We need to treat the arrays as matrices, which numpy will not do automatically
%timeit x = np.linalg.inv(A) @ b

In [ ]:
# The speedup of np.linalg.solve() vs inversion and multiplication should be, from theory a little larger than a factor of 2, which is
# about what we see here.  If you need to solve a system 3 times with a different value of b, but the same value of A, use inversion.
%timeit for i in range(5): np.linalg.solve(A,b)

In [ ]:
# The inversion is the only expensive step, each additional iteration is effectively free
print("inverting matrix...")
%timeit A_inv = np.linalg.inv(A)
print("done")
A_inv = np.linalg.inv(A) 
print("solving 5 times...")
%timeit  for i in range(5): A_inv @ b
print("done")

# Solving Nonlinear Equations on computers
Computers can't solve nonlinear equations :(

jk, while computers can't solve nonlinear equations directly, almost all modern computational physics (including numerical relativity)
are about the study of how to approximate the solutions to nonlinear equations on computers


By far the most common strategy for solving nonlinear equations is to approximate them by linear equations and solve the linear equations.  The process of approximating a nonlinear function by a linear function is called *calculus* :)

The simplest example of solving a nonlinear equation this way was invented by one of the originators of calculus: Newton's method.
Newton's method approximates a function $f(x)$ by it's derivative and solves an equation using a linear approximation to the function
rather than the function itself.  If we have an initial guess $x_0$ for the solution to an equation, i.e.
$$f(x) = 0$$
We note that the guess is likely not perfect, so $f(x_0) \neq 0$.  What we do is we build a linear function that approximates $f(x)$ near $x_0$
$$f_1(x) = f'(x_0)x  + f(x_0)$$
This function has the same value and slope as $f(x)$ at $x=x_0$, therefore it's probably close to the function so long as $x$ is close to $x_0$.  
We instead find the zero of this function
$$f_1(x) = f'(x_0)(x-x_0) + f(x_0) = 0$$
Which we can solve using the tools above
$$x_1 = -f(x_0)/f'(x_0) + x_0 $$
We label the solution of this equation $x_1$, now note $f(x_1)$ is probably not equal to zero, but it is likely closer than $f(x_0)$

In [ ]:
# A demo of this algorithm in practice
# The function and it's derivative are known analytically
def f(x):
    return np.sin(x) - 0.652
def f_prime(x):
    return np.cos(x)
# choose some initial guess
x_0 = 1.2 * np.pi/4
# set the current guess to the initial guess
x_guess = x_0
# Make a plot for each iteration
N_iters = 5
fig, axs = plt.subplots(N_iters, 1, sharex=True,figsize=(6, 24))

x_vals = np.linspace(np.pi/6,  np.pi/3)
for i in range(N_iters):
    # Plot the function
    axs[i].plot(x_vals, np.sin(x_vals) - 0.652, color="navy", label=r"$f(x)$")
    # Plot line x=0
    axs[i].plot(x_vals, np.zeros_like(x_vals))
    # Plot the x-value of the guess
    axs[i].axvline(x_guess, color="red")
    # Plot the approximating function
    axs[i].plot(x_vals, f(x_guess) + f_prime(x_guess) * (x_vals-x_guess), color="deepskyblue", label=rf"$f_{i+1}(x)$")
    axs[i].legend()
    # Update the guess
    x_guess = -f(x_guess)/f_prime(x_guess) + x_guess
plt.legend()    
plt.plot()
print("final estimate x_guess = ", x_guess)
print("function value f(x_guess) = ", f(x_guess))

After only 3 iterations the location of the aproximate zero is already exquisitely good, after 5 iterations the zero is found to greater than machine precision.  When Newton's method converges, it *superconverges*: the number of correct digits doubles with each iteration.


Many Nonlinear problems are solved in a similar manner, optimization is a highly related topic which is now very much in vogue, and one such 
algorithm for optimization is called *gradient descent*, which is nearly identical to Newton's method.  Modern approaches typically build on 
these methods to make them more robust and get them to converge faster, in general you should use a nonlinear solver from a library, such 
as `scipy.optimize.root`, which takes a function $f(x)$ (possibly vector valued), and an initial guess $(x_0)$ and returns a root

In [ ]:
#This function takes a function callable (i.e. a function) and an initial guess, and some other optional arguments 
# (including the jacobian which in this case is just the derivative) we don't need to supply the jacobian, but it can improve 
# performance if we know it (otherwise it has to construct it numerically by calling the function many times)
?scipy.optimize.root

In [ ]:
# The main thing to notice is the "x"-value which is output (along with a bunch of other useful information)
# You'll see it agrees with the value we found above
found_root = scipy.optimize.root(f, x0 = x_0, jac=f_prime)
print(found_root)
print("x = ", found_root.x)

In [ ]:
# This also works
scipy.optimize.root(f, x0 = x_0)

## Caution!
If a function has multiple roots, there's no guarantee that Newton's method will converge to any particular root.  If 
you need a *specific* root of a function, you may have to investigate other numerical methods, or place bounds on the domain
you are searching for the solution in.


# Solving Differential Equations on computers
Computers can't solve differential equations :((

Well ok, maybe Wolfram Alpha can, but in general there will be differential equations sufficiently complicated to not have
analytic solutuions.  In these cases, the solution to the differnetial equation must be constructed by approximating the problem
of solving the differntial equation by a solving a sequence of linear equations. Typically the solution to a differential equation 
is constructed via a sequence of steps, where the function is approximated by it's derivative (sound familiar?).  Although exactly how 
this is done can substantially change how quickly and accurately the differntial equation is solved.  In general if an ODE solver seems to be taking a long time, google the word "Stiff" and try a different  solver.  

For example, if we want to solve an ordinary differential equation of the form
$$y'(t) = f(t,y)$$
With an initial condition $y(t_0) = y_0$
the best thing to do is to use an implemented ode solver.  We'll use an implementation from the 
`scipy.integrate` library.  This is an initial value problem (IVP), so we will use 
`scipy.integrate.solve_ivp`

In [ ]:
?scipy.integrate.solve_ivp

By default `solve_ivp` uses a particular numerical method `RK45` to solve the differntial equation.  This is a 4th order Runge-Kutta method 
(which means steps are taken by effectively taylor expanding a function to 4th order) and errors are controlled by comparing to a 5th
order method.  This is a good default to use, but in certain cases (stiff systems), a different solver should be used.  It's often not 
possible to know what solver will work before trying it.  

In [ ]:
# Well solve a simple just-barely-nonlinear differential equation
def f(t, y):
    return -y + 0.1 * y**3
# We'll want to solve the equation over some range
# almost always, we'll want to use 'dense output' which means
# that we'll ask the solver to output the solutuon to the ODE 
# at intermediate times between the initial and final time.  
# we specify which times this will be by using the `t_eval` argument

# Where we solve over
t_span = (0, 3.0)
# Where we want to output the solution at  
# (the * notation means to expand the tuple and )
# and treat it as arguments to a function
t_eval = np.linspace(*t_span, 30)

# Just for making coloring lines easily
import matplotlib.cm as cm
for i, y_0  in enumerate([1.0, 2.0, 3.0, 3.1, 3.15, 3.18]):
    # This is where the ODE is actually solved
    solution = scipy.integrate.solve_ivp(fun=f, t_span=t_span, y0=np.array([y_0]), dense_output=True, t_eval=t_eval)
    plt.plot(solution.t, solution.y[0,:], label=f"y_0 = {y_0}", color=cm.viridis(i/5))
plt.ylabel("y(t)")
plt.xlabel("t")
plt.legend(loc="upper left")
    

## Challenge Problem
Use `solve_ivp` to find the solution to a general 1-d Newtonian dynamics problem. That is, assume there is an object with mass  $m$ 
moving under the influence of some net force $F$, with a velocity $v = dx/dt$ (say it starts out with an initial velocity $v_0$, at an initial position $x_0$)
Hint: You'll need to define a *state* vector $Z = (x(t), v(t))$, and give a vector equation for the evolution of the state to the  IVP solver.  I.e. you'll need
$$\frac{dZ}{dt} = \left(\frac{dx}{dt}, \frac{dv}{dt}\right)$$
And you'll need to express $dx/dt$ and $dv/dt$ in terms of the state variables $(t, x, v)$, and $F(t,x,v)$ the force


In [ ]:
def dZ_by_dt(t, Z, F):
    x = Z[0]
    v = Z[1]
    F_applied = F(t, x, v)
    # TODO your code here
    return None

def F_spring(t, x, v):
    return -x

# Define variables needed for the solve_ivp call


# Below, the syntax "lambda" just defines a function without a name and returns it (called an anonymous function)
# It makes it easier to make functions more generic when defined so later values cal be substituted into them
# Note lambdas automatically return, but do not explicitly use the `return` keyword

solution_Z = scipy.integrate.solve_ivp(lambda t, Z: dZ_by_dt(t, Z, F_spring), ) # Complete this function call with your own code

# Make a plot of the solution!

## Double challenge problem
Write code to solve an N-d Newton problem (i.e. a newtonian mechanics problem with N degrees of freedom)